# Exploring Hacker News Posts

In this project, we compare two different types of posts from [Hacker News](https://news.ycombinator.com/).
Hacker News is a site started by the startup incubator Y Combinator, where user-submitted technology related stories (known as "posts") are voted and commented upon, similar to reddit. 

We are specifically interested in posts whose titles begin with either _Ask HN_ or _Show HN_. Users submit _Ask HN_ posts to ask the Hacker News community a specific question. Likewise, users submit _Show HN_ posts to show the Hacker News community a project, product, or just generally something interesting.

We specifically compare these two types of posts to determine the following:
1. Do _Ask HN_ or _Show HN_ receive more comments on average?
2. Do posts created at a certain time receive more comments on average?

The [dataset](https://www.kaggle.com/hacker-news/hacker-news-posts#) has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Refer the dataset **hacker_news.csv** available in the repository.


## *Step 1*: Extracting the dataset as a list of lists

In [1]:
from csv import reader
opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

def explore(table):
    for i in range(5):
        print(table[i])
        print('\n')

explore(hn)    

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']




We remove the header and store it in a seperate variable.

In [2]:
header = hn[0]
hn = hn[1:]
print(header, '\n')
explore(hn)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'] 

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




## *Step 2*: Extracting _Ask HN_ and _Show HN_ posts
Since we are only interested in post titles beginning with _Ask HN_ or _Show HN_ we will create new lists of lists containing just the data for those titles.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Ask HN", len(ask_posts))        
print("Show HN", len(show_posts))        
print("Others", len(other_posts))        

Ask HN 1744
Show HN 1162
Others 17194


## *Step 3*: Calculating average number of comments for _Ask HN_ and _Show HN_ posts
We determine whether ask posts or show posts recieve more comments on average.

In [27]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
avg_ask_comments = total_ask_comments/len(ask_posts) 
print("Average ask comments", avg_ask_comments)

total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
avg_show_comments = total_show_comments/len(show_posts) 
print("Average show comments", avg_show_comments)

total_other_comments = 0
for row in other_posts:
    num_comments = int(row[4])
    total_other_comments += num_comments
avg_other_comments = total_other_comments/len(other_posts) 
print("Average other comments", avg_other_comments)

Average ask comments 14.038417431192661
Average show comments 10.31669535283993
Average other comments 26.8730371059672


Ask post recieve more comments on average. 
Recall that we are more interested in ask and show posts. Other posts have been analyzed in the additional section in the end.
Since ask posts are more likely to recieve comments, we will focus our remaining analysis on these posts.


## *Step 4*: Calculating the average number of comments for _Ask HN_ posts by hour
Next, we determine if ask posts created at a certain time are more likely to attract comments.

In [17]:
import datetime as dt

result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"
for row in result_list:
    date = row[0]
    comments = row[1]
    time = dt.datetime.strptime(date, date_format)
    time = time.strftime("%H")
    if time in counts_by_hour:
        counts_by_hour[time] += 1
        comments_by_hour[time] += comments
    else:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comments
        
print(counts_by_hour)
print(comments_by_hour)


{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}
{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


We will use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

In [18]:
avg_by_hour = []
for hour in comments_by_hour:
    avg = comments_by_hour[hour]/counts_by_hour[hour]
    avg_by_hour.append([hour, avg])
    
print(avg_by_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


## *Step 5*: Sorting and printing the values
Making the result more understandable by sorting the list and displaying the top 5 results.

In [25]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse = True)

print("Top 5 hours for Ask HN post comments")
template = "{time} - {avg:.2f} average comments per post"
for row in sorted_swap[:5]:
    print(template.format(time = dt.datetime.strptime(row[1], "%H").strftime("%H:%M"), avg = row[0]))
    

Top 5 hours for Ask HN post comments
15:00 - 38.59 average comments per post
02:00 - 23.81 average comments per post
20:00 - 21.52 average comments per post
16:00 - 16.80 average comments per post
21:00 - 16.01 average comments per post


## Conclusion:
Note that the data set we analyzed excluded posts without any comments. Given that, of the posts that received comments, ask post received more comments on average, and ask posts created between 15:00 and 16:00 received most comments on average.

## *Additional*: Analyzing other posts
After creating the __other_posts__ list, we repeat the steps followed for __ask_posts__ for other posts.
Note that this analysis will only show us a general trend about when most users comment on posts. However, we can not determine the nature of these posts. 

In [32]:
import datetime as dt

other_result_list = []
for row in other_posts:
    created_at = row[6]
    num_comments = int(row[4])
    other_result_list.append([created_at, num_comments])

other_counts_by_hour = {}
other_comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"
for row in other_result_list:
    date = row[0]
    comments = row[1]
    time = dt.datetime.strptime(date, date_format)
    time = time.strftime("%H")
    if time in other_counts_by_hour:
        other_counts_by_hour[time] += 1
        other_comments_by_hour[time] += comments
    else:
        other_counts_by_hour[time] = 1
        other_comments_by_hour[time] = comments
        
print(other_counts_by_hour, '\n')
print(other_comments_by_hour, '\n')

other_avg_by_hour = []
for hour in other_comments_by_hour:
    avg = other_comments_by_hour[hour]/other_counts_by_hour[hour]
    other_avg_by_hour.append([avg, hour])
    
print(other_avg_by_hour, '\n')
other_sorted_swap = sorted(other_avg_by_hour, reverse = True)

print("Top 5 hours for other post comments")
template = "{time} - {avg:.2f} average comments per post"
for row in other_sorted_swap[:5]:
    print(template.format(time = dt.datetime.strptime(row[1], "%H").strftime("%H:%M"), avg = row[0]))
    

{'11': 660, '19': 980, '22': 758, '00': 611, '04': 454, '09': 534, '16': 1101, '18': 1084, '10': 591, '12': 789, '20': 911, '03': 407, '17': 1169, '14': 958, '13': 918, '01': 500, '23': 674, '08': 496, '02': 441, '21': 874, '15': 1040, '06': 408, '05': 388, '07': 448} 

{'11': 19532, '19': 26167, '22': 17635, '00': 16544, '04': 10953, '09': 14732, '16': 27959, '18': 29186, '10': 15728, '12': 23944, '20': 21080, '03': 10918, '17': 32727, '14': 30973, '13': 28363, '01': 11536, '23': 16592, '08': 13405, '02': 12254, '21': 20635, '15': 30700, '06': 8714, '05': 9768, '07': 12010} 

[[29.593939393939394, '11'], [26.701020408163266, '19'], [23.265171503957784, '22'], [27.076923076923077, '00'], [24.125550660792953, '04'], [27.588014981273407, '09'], [25.394187102633968, '16'], [26.924354243542435, '18'], [26.612521150592215, '10'], [30.34727503168568, '12'], [23.13940724478595, '20'], [26.825552825552826, '03'], [27.99572284003422, '17'], [32.33089770354906, '14'], [30.896514161220043, '13'],